In [1]:
#Para automatizar el proceso se configura el nombre del archivo dado el mes y el año actual
import datetime 
    
current_time = datetime.datetime.now() 
anio=(current_time.year)-2000
mes=current_time.month-2

print (anio)      
print (mes) 
     

22
9


In [2]:
# Importación de librerías
import pandas as pd
import numpy as np

In [3]:
!ls Documentos/FileServer

'ACARA 9-22.csv'


In [4]:
# Se asume que el archivo es dejado en una ubicación específica, como por ejemplo FileServer (podría ser un File System)
archivo = ('Documentos/FileServer/ACARA '+str(mes)+'-'+str(anio)+'.csv')

In [5]:
# Se carga csv
df = pd.read_csv(archivo, sep=';')

In [6]:
#Identificación de las columnas
df.columns

Index(['mtm', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015',
       '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006',
       '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998'],
      dtype='object')

In [7]:
# últimas 5 filas
df.tail(5)

,mtm,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998
19902,00507CH,NaN,NaN,NaN,NaN,160000.0,148800.0,138400.0,128700.0,119700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19903,20519022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110000.0,102300.0,...,61600.0,57200.0,53200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19904,20519079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110000.0,102300.0,...,61600.0,57200.0,53200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19905,20519076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110000.0,102300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19906,20519069,NaN,NaN,NaN,NaN,160000.0,148800.0,138400.0,128700.0,119700.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#Se transforma el dataframe
df_prefinal = df.melt(id_vars=["mtm"], value_name='precio', var_name='anio').sort_values(by=["mtm", "anio"])

In [9]:
#primeras 5 filas
df_prefinal.head(5)

,mtm,anio,precio
481120,0010103,1998,NaN
461213,0010103,1999,NaN
441306,0010103,2000,NaN
421399,0010103,2001,NaN
401492,0010103,2002,NaN


In [10]:
#Se filtran las filas con valores nulos
final=df_prefinal[~pd.isnull(df_prefinal).any(axis=1)]

In [11]:
final.head(5)

,mtm,anio,precio
282050,0010103,2008,8600.0
262143,0010103,2009,9900.0
242236,0010103,2010,11100.0
222329,0010103,2011,12300.0
202422,0010103,2012,13600.0


In [12]:
#Se eliminan duplicados
final = final.drop_duplicates()

In [14]:
import mysql.connector
# Conexión MySQL 
conection = mysql.connector.connect(user='root', host='localhost', password='tupassword', database='Acara',  auth_plugin='mysql_native_password')

cursor1=conection.cursor()

# Se crea la tabla
cursor1.execute("CREATE TABLE IF NOT EXISTS acara_precios_mensual (mtm_id VARCHAR(255), anio INT, precio FLOAT, actualizacion timestamp);")
print("se creo la tabla")

# Se insertan los datos
for index, row in final.iterrows():
    cursor1.execute("INSERT INTO acara_precios_mensual (mtm_id, anio, precio, actualizacion) VALUES (%s, %s, %s, DATE(NOW()))", (row['mtm'], row['anio'], row['precio']))

conection.commit()
print("se insertaron los datos")
cursor1.close()
conection.close()


se creo
se pasaron los datos
